# Create Connection To Postgres

In [41]:
import pandas
import numpy
from connect_sql_db import build_engine

In [2]:
engine = build_engine(database_name="database1",host="35.225.193.21")

# Read in Yelp Data

In [3]:
#read in business data
busi_df = pandas.read_json("../data/yelp_dataset/original_json/yelp_academic_dataset_business.json", lines = True)

In [7]:
#save as csv
busi_df.to_csv("../data/yelp_dataset/yelp_csvs/yelp_business_raw.csv",index = False)

In [4]:
#read in review data
review_df = pandas.read_json("../data/yelp_dataset/original_json/yelp_academic_dataset_review.json", lines = True)

In [8]:
#save as a csv
review_df.to_csv("../data/yelp_dataset/yelp_csvs/yelp_review_raw.csv", index = False)

# Python SQL Functions

In [82]:
#return yelp dataframes dictionary from sql
def read_in_tables(table_dict):
    return_me = {}
    for name,dataframe in table_dict.items():
        return_me[name] = pandas.read_sql(f'select * from "{dataframe}"',con=engine)
        
    return return_me

#send dataframe dictionary to sql
def send_in_dataframes(tables_dict):
    for name,dataframe in tables_dict.items():
        dataframe.to_sql(
            f"{name}", 
            con = engine,
            index = False, 
            method = 'multi',
            chunksize = 1000,
            if_exists = "append"
        )
        print(f"{name} df sent...")
    return "dataframe data sent"

#drop dataframe dictionary from sql
def drop_dataframes(tables_dict):
    with engine.connect() as connection:
        for name in tables_dict.keys():
            connection.execute(f'DROP TABLE "{name}"')
            print(f"{name} dropped...")
    return "tables dropped"

# ETL PROCESS

In [37]:
#inspect dtypes
busi_df.dtypes

business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
res                bool
dtype: object

In [38]:
#inspect dtypes
review_df.dtypes

review_id              object
user_id                object
business_id            object
stars                   int64
useful                  int64
funny                   int64
cool                    int64
text                   object
date           datetime64[ns]
dtype: object

In [12]:
#check dfs
busi_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [13]:
#check dfs
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


In [14]:
#collect all possible categories
categories = busi_df.categories.tolist()

categories_lst = []
for i in categories:
    if i != None:
        for v in i.split(","):
            categories_lst.append(v)


In [15]:
#view unique categories
set(categories_lst)

{'Windshield Installation & Repair',
 ' Occupational Therapy',
 ' Structural Engineers',
 ' Registry Office',
 'Auto Repair',
 'Pressure Washers',
 'Asian Fusion',
 'Bars',
 ' Cannabis Collective',
 'Fire Departments',
 ' Mediators',
 ' Indoor Playcentre',
 ' Livestock Feed & Supply',
 'Massage Therapy',
 'Lebanese',
 ' Fire Protection Services',
 ' Party Bike Rentals',
 ' Professional Sports Teams',
 'Discount Store',
 ' Nanny Services',
 'Personal Care Services',
 ' Hookah Bars',
 ' Dive Shops',
 'Art Schools',
 ' Playgrounds',
 'Bartenders',
 ' Pasta Shops',
 ' Reflexology',
 'Mortuary Services',
 'Digitizing Services',
 'Afghan',
 'Muay Thai',
 ' Golf Cart Rentals',
 ' Generator Installation/Repair',
 'Ethnic Food',
 'Colombian',
 ' Trampoline Parks',
 'Livestock Feed & Supply',
 'Religious Items',
 ' Paint & Sip',
 'Hypnosis/Hypnotherapy',
 'Kosher',
 ' Distilleries',
 ' Donation Center',
 ' Convenience Stores',
 'Title Loans',
 'Grout Services',
 ' Windows Installation',
 'Sushi 

In [16]:
#searching for categories that include restaurants
for i in busi_df.categories.tolist():
    if i != None:
        if "restaurants" in i.lower().strip():
            print(i)

Ethnic Food, Food Trucks, Specialty Food, Imported Food, Argentine, Food, Restaurants, Empanadas
Food, Restaurants, Grocery, Middle Eastern
Restaurants, Cheesesteaks, Poutineries
Japanese, Fast Food, Food Court, Restaurants
Persian/Iranian, Turkish, Middle Eastern, Restaurants, Kebab
Food, Pretzels, Bakeries, Fast Food, Restaurants
Mexican, Restaurants, Fast Food
Restaurants, Burgers, Food
Restaurants, Vietnamese, Soup
Burgers, Restaurants, American (Traditional), Chicken Wings
Restaurants, Lebanese, Middle Eastern
Mexican, Restaurants
Burgers, Fast Food, American (New), Restaurants, Hot Dogs
Restaurants, Fast Food, Burgers
Chinese, Restaurants
Shopping Centers, Food, Coffee & Tea, Cafes, Museums, Restaurants, Shopping, Local Flavor, Flowers & Gifts, Arts & Entertainment, Art Galleries, Florists
Restaurants, Egyptian, Food Trucks, Food, Middle Eastern
Fast Food, American (New), Restaurants, Burgers
Restaurants, Pizza
Vietnamese, Restaurants
Restaurants, Buffets, Indian, Halal
Breakfast

Breakfast & Brunch, Restaurants
Arts & Entertainment, Nightlife, Breakfast & Brunch, Music Venues, Bars, Mexican, Restaurants
Italian, Comfort Food, Asian Fusion, Restaurants, Noodles, Thai, Specialty Food, Salad, Food, Pasta Shops
Italian, Chicken Wings, Restaurants, Pizza
Restaurants, Burgers, Food, Fast Food
Food, Restaurants, Southern, Cafes, Coffee & Tea
Bubble Tea, Cafes, Food, Restaurants
Lebanese, Middle Eastern, Restaurants
Food, Beer, Wine & Spirits, Wine Bars, Nightlife, French, Bars, Restaurants, Specialty Food, Breakfast & Brunch
Chinese, Restaurants, Dim Sum, Beauty & Spas
Pizza, Sandwiches, Restaurants, Italian
Restaurants, Pizza, Chicken Wings, Sandwiches, Fast Food
Sandwiches, Fast Food, Restaurants, Burgers
Restaurants, Greek
Bars, Salad, Wine Bars, Nightlife, Italian, Restaurants, Soup
Italian, Restaurants
Cafes, Restaurants, Coffee & Tea, Food
Barbeque, Pubs, Nightlife, Indian, Szechuan, Restaurants, Bars, Chinese, Karaoke
Restaurants, Nightlife, Karaoke
Bakeries, B

American (Traditional), Restaurants, French
Restaurants, Tex-Mex
Burgers, Restaurants, Fast Food
Nightlife, Beer Bar, Breakfast & Brunch, Restaurants, American (Traditional), Bars, Sports Bars, Pizza, Pubs
Restaurants, Mexican, Fast Food
Restaurants, Chinese
Tapas/Small Plates, Restaurants, Cafes, Specialty Food, Mediterranean, International Grocery, Food, Imported Food
Restaurants, Italian, Pizza
Mexican, Restaurants
Asian Fusion, Fast Food, Chinese, Restaurants, Dim Sum
Vietnamese, Restaurants
Coffee & Tea, Coffee Roasteries, Food, Sandwiches, Restaurants, Breakfast & Brunch
Greek, Restaurants
Chinese, Restaurants, Noodles
Jazz & Blues, Cafes, Nightlife, Restaurants, Sandwiches, Arts & Entertainment
Restaurants, Thai
Chinese, Restaurants, Caterers, Event Planning & Services
Restaurants, Mexican, Food, Salad, Desserts
Pizza, Restaurants, Chicken Wings, Sandwiches
Fast Food, Restaurants, Tacos, Tex-Mex, Mexican
Ice Cream & Frozen Yogurt, Food, Chicken Wings, Food Trucks, Restaurants, H

Food, Meat Shops, Restaurants, Event Planning & Services, Cafes, Specialty Food, Caterers, Delis, Sandwiches
Restaurants, Modern European, Spanish
Pizza, Chicken Wings, Sandwiches, Restaurants
American (New), Restaurants
Restaurants, Thai, Asian Fusion, Sushi Bars
Restaurants, Specialty Food, Food, Meat Shops
Breakfast & Brunch, American (Traditional), Restaurants, Diners
Restaurants, Event Planning & Services, Nightlife, Barbeque, Food Trucks, Beer Bar, Caterers, Food, Bars
Mexican, Latin American, Restaurants
Restaurants, Delis, Sandwiches
Food, Restaurants, Coffee & Tea, Cafes
Restaurants, Coffee & Tea, Burgers, Food, Fast Food
Restaurants, Bars, Cocktail Bars, Nightlife, American (Traditional), Food, Desserts
Fast Food, Restaurants, Chinese
Diners, Chinese, Restaurants
Indian, Restaurants, Pakistani, Middle Eastern
Specialty Food, Mexican, Sandwiches, Food, Ethnic Food, Salad, Restaurants
Chicken Wings, Pizza, Restaurants, Buffets
Cafes, Restaurants, Soul Food, American (Traditiona

French, Restaurants
Vietnamese, Thai, Chinese, Restaurants, Asian Fusion
Bagels, Food, Restaurants, Breakfast & Brunch, Bakeries
Soup, Food, Breakfast & Brunch, Salad, Bagels, Coffee & Tea, Restaurants, Sandwiches
Chinese, Restaurants
Food, Bakeries, Italian, Restaurants
Italian, Pizza, Restaurants
Sushi Bars, Restaurants, Japanese
Restaurants, Nightlife, Pizza, Italian, Beer Bar, Diners, Bars, Desserts, Wine Bars, Food
Restaurants, Italian, Pizza, Chicken Wings
Restaurants, Korean, Sushi Bars
Beer Bar, Restaurants, Beer, Wine & Spirits, Pubs, Bars, Local Flavor, Food, Nightlife, Breweries
Fast Food, Chinese, Restaurants
Restaurants, Bars, American (Traditional), Barbeque, Vegetarian, Nightlife
Sandwiches, Restaurants, Pizza
Chinese, Restaurants, Wraps, Chicken Wings
Indian, Vegetarian, Restaurants, Vegan
Restaurants, Vietnamese
Restaurants, Chicken Wings, Food
Buffets, Teppanyaki, Japanese, Restaurants, Steakhouses, Sushi Bars, Cajun/Creole, Asian Fusion
Bars, Nightlife, American (New

...

Pizza, Restaurants, Italian
Burgers, Restaurants, Steakhouses, Chicken Wings, Bars, American (Traditional), Nightlife, Sports Bars
Pizza, Italian, Salad, Sandwiches, Restaurants
Burgers, Fast Food, Coffee & Tea, Food, Restaurants
Restaurants, Mediterranean
Burgers, Fast Food, Coffee & Tea, Restaurants, Food
Sushi Bars, Japanese, Restaurants, Hawaiian, Food, Poke
Mexican, Tex-Mex, Restaurants
Restaurants, Modern European, Mediterranean, Middle Eastern, Kebab
Mexican, Restaurants
Cambodian, Restaurants, Sandwiches, Caribbean, Comfort Food
Gluten-Free, Food Delivery Services, Salad, Food, Restaurants, Sandwiches, Vegetarian
Restaurants, Chinese
Restaurants, Japanese
Food, Mexican, Food Trucks, Middle Eastern, Restaurants
Restaurants, Internet Cafes, Food, Sandwiches, Coffee & Tea
Food, German, Desserts, Fish & Chips, Restaurants
Vietnamese, Korean, Restaurants, Thai
Sandwiches, Restaurants, Fast Food
Restaurants, Steakhouses, Seafood
Caribbean, Restaurants
Specialty Food, Ethnic Food, Foo

Restaurants, Caribbean
Barbeque, Restaurants, Steakhouses
Restaurants
Indian, Restaurants
Restaurants, Pizza
Restaurants, Pizza
Restaurants, Korean
Seafood, Restaurants, Chicken Wings
Restaurants, Ethnic Food, Ethiopian, Specialty Food, Vegan, Food, Vegetarian
Fast Food, Burgers, Restaurants, Food, Coffee & Tea
Burgers, Hot Dogs, Restaurants, Fast Food
Pizza, Breakfast & Brunch, Coffee & Tea, Restaurants, Canadian (New), Food
Italian, Restaurants
Restaurants, Sandwiches, Gluten-Free, Latin American
French, Restaurants, Brasseries
Restaurants, Caterers, Italian, Event Planning & Services
Restaurants, Gluten-Free, Pizza
Restaurants, Italian
Pizza, Restaurants, Italian, Chicken Wings
Restaurants, Desserts, Vegetarian, Food, Indian
Food, Juice Bars & Smoothies, Acai Bowls, Ice Cream & Frozen Yogurt, Gluten-Free, Restaurants
Food, Live/Raw Food, Juice Bars & Smoothies, Breakfast & Brunch, American (New), Acai Bowls, Kombucha, Vegetarian, Restaurants, Coffee & Tea
Restaurants, Buffets, Ameri

Cajun/Creole, Restaurants
Pizza, Restaurants
Dim Sum, Korean, Sushi Bars, Restaurants, Seafood, Chinese, Buffets, Japanese
Restaurants, Pizza, Fast Food
Restaurants, Cafes, Sandwiches, Salad
Breakfast & Brunch, Coffee & Tea, Restaurants, Donuts, Food
Breakfast & Brunch, Restaurants
Restaurants, Italian
Restaurants, Thai, Food
Restaurants, Sushi Bars, Poke, Food
Imported Food, Ethnic Food, Food, Chinese, Restaurants, Specialty Food
Mexican, Restaurants
Restaurants, Thai
Active Life, Golf, Restaurants
Sushi Bars, Restaurants, Japanese
Restaurants, Breakfast & Brunch, Creperies, Fondue
Sushi Bars, Japanese, Restaurants
Burgers, American (New), Breweries, Food, Restaurants, Bars, American (Traditional), Nightlife
Restaurants, Chinese, Food, Ethnic Food, Imported Food, Specialty Food, Vietnamese, Thai
Desserts, Juice Bars & Smoothies, Restaurants, Food, Ice Cream & Frozen Yogurt, Fast Food, Burgers
Restaurants, Caribbean, Latin American
Restaurants, Sandwiches, Delis, Steakhouses
Restaurant

Restaurants, American (Traditional)
Restaurants, Sushi Bars
Restaurants, Asian Fusion, Chinese
Restaurants, Pizza, Arts & Entertainment, Active Life, Indoor Playcentre, Arcades
Korean, Restaurants
Food, Restaurants, Barbeque, Fast Food
Bagels, Food, Restaurants, Breakfast & Brunch, Sandwiches
Restaurants, Tacos, Sandwiches, Mexican, Seafood, New Mexican Cuisine
Restaurants, Asian Fusion, Japanese, Food
Buffets, Restaurants
Japanese, Restaurants
Middle Eastern, Restaurants
Restaurants, Pizza
Restaurants, American (New)
Fast Food, Restaurants
Restaurants, Mediterranean
Pubs, Irish Pub, Bars, Irish, Nightlife, Soup, Restaurants
Restaurants, Pizza
Restaurants, Food, Coffee & Tea, Breakfast & Brunch, Cafes
Restaurants, Chinese, Japanese
Food, Breweries, Venues & Event Spaces, Restaurants, Pizza, Event Planning & Services
Bakeries, Desserts, Food, Restaurants, Cupcakes, Delis
Nightlife, Bars, American (Traditional), Restaurants, Tex-Mex
Restaurants, Thai
Pubs, Nightlife, American (Traditiona

Thai, Restaurants
Event Planning & Services, Music Venues, Cajun/Creole, Haitian, Caterers, Restaurants, French, Arts & Entertainment, Party & Event Planning, Caribbean, Nightlife
Restaurants, Pizza
Restaurants, Noodles
Chinese, Restaurants, Bakeries, Desserts, Food
Coffee & Tea, Food, Arts & Entertainment, Restaurants, Shopping, Vegan, Art Galleries
Restaurants, Mexican, Nightlife, Bars
American (New), Restaurants, Breakfast & Brunch, Salad, Sandwiches, Food, Coffee & Tea
Greek, Mediterranean, Restaurants
Afghan, African, Restaurants
Restaurants, Fitness & Instruction, Cafes, Golf Lessons, Golf, Active Life
Restaurants, Sports Bars, American (New), Nightlife, Bars
Food, Bars, Canadian (New), Nightlife, Restaurants, Cocktail Bars
Restaurants, Food, Noodles, Japanese, Ramen
Restaurants, Mediterranean, Vegan, American (New), Mexican
Food, Restaurants, Pizza, Donairs, Mediterranean
Asian Fusion, Restaurants, Cantonese, Noodles, Szechuan, Chinese
Restaurants, Eyelash Service, Beauty & Spas

In [17]:
#function returning boolean value if `restaurants` exists in the given string
def contains_restaurant(x):
    if x != None:
         if "restaurants" in x.lower().strip():
                return True
   
    return False


#create a new column to hold the boolean values
busi_df["res"] = busi_df.categories.apply(lambda x: contains_restaurant(x))

In [58]:
#create a df where restaurants value is True
new_busi_df = busi_df.loc[busi_df.res == True].copy()

In [59]:
#drop invalid columns
new_busi_df.drop(["attributes","categories","hours","res"], axis=1, inplace = True)

In [60]:
new_busi_df.replace("", numpy.nan, inplace = True)

In [67]:
new_busi_df.dropna(inplace=True)

In [71]:
new_review_df = pandas.merge(
    new_busi_df[["business_id"]],
    review_df,
    how = "left",
    on = "business_id"    
)

In [72]:
new_review_df.replace("", numpy.nan, inplace = True)

In [73]:
new_review_df.dropna(inplace=True)

# Send New DataFrames To Postgre

In [74]:
#create dictionary of cleaned dataframes
yelp_dfs = {
    "business_table": new_busi_df,
    "review_table": new_review_df
}

In [ ]:
send_in_dataframes(yelp_dfs)

business_table df sent...


In [80]:
print(len(new_busi_df),len(new_review_df))

63438 5044970
